In [1]:
# mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
# go to your code files directory
import os
os.chdir("/content/drive/My Drive/Earth-surface-water-mapping")
# !ls
# !nvidia-smi

Mounted at /content/drive/


In [2]:
try:
    get_ipython().magic(u'tensorflow_version 2.x')
except Exception:
    pass
import tensorflow as tf
import numpy as np
from utils.utils import readTiff

In [ ]:
TrainScenes = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/TrainingScene/*.tif'
TrainTruths = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/TrainingTruth/*.tif'
record_file = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/traData.tfrecords'

scene_paths = sorted(tf.io.gfile.glob(TrainScenes))
truth_paths = sorted(tf.io.gfile.glob(TrainTruths))
traData_paths = list(zip(scene_paths,truth_paths))

def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

# Create a dictionary with features that may be relevant.
def image_example(image,truth):
    feature = {
        'bandShape': int64_feature(image[:,:,0].shape),
        'blueBand': float_feature(image[:,:,0].flatten()),
        'greenBand': float_feature(image[:,:,1].flatten()),
        'redBand': float_feature(image[:,:,2].flatten()),
        'nirBand': float_feature(image[:,:,3].flatten()),
        'swir1Band': float_feature(image[:,:,4].flatten()),
        'swir2Band': float_feature(image[:,:,5].flatten()),
        'truth': float_feature(truth.flatten()),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

# Write to a `.tfrecords` file.
with tf.io.TFRecordWriter(record_file) as writer:
    for scenePath, truthPath in traData_paths:
        print(scenePath)
        scene, _, _, _, _, _ = readTiff(scenePath)
        truth, _, _, _, _, _ = readTiff(truthPath)
        scene = np.clip(scene/10000,0,1)
        tf_example = image_example(scene, truth)
        writer.write(tf_example.SerializeToString())
